# Curso de culminación: Creación de un proyecto de ML de datos tabulares de extremo a extremo con SageMaker Studio y el SDK para Python de Amazon SageMaker

## Configuración del entorno

Este código de configuración básica se ha incluido para ayudarlo a comenzar. Lea y ejecute primero estas celdas para instalar los paquetes y crear las variables.

In [ ]:
%%capture
%pip install -U shap
%pip install -U smdebug
%pip install imbalanced-learn
%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install sagemaker
%pip install -U seaborn

In [ ]:
#required-libraries

import boto3
import datetime as datetime
import io
import IPython
import json
import math
import matplotlib.pyplot as plt  # visualization
import numpy as np
import pandas as pd
import pathlib
import re
import sagemaker
import seaborn as sns  # visualization
import statistics
import string
import sys
import time
import zipfile

from imblearn.over_sampling import SMOTE

from sagemaker import clarify
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition
from sagemaker.debugger import CollectionConfig, DebuggerHookConfig, FrameworkProfile, ProfilerConfig, ProfilerRule, Rule, rule_configs
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TransformInput
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.network import NetworkConfig
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.steps import TuningStep
from sagemaker.xgboost.estimator import XGBoost
from sklearn.model_selection import train_test_split

In [ ]:
#basic-variable-code-and-settings

%matplotlib inline
base_job_name = "capstone-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "sagemaker/capstone"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
save_interval = 5
sns.set(color_codes=True)

## Descripción del conjunto de datos

Las siguientes cinco tablas se almacenan en un bucket de Amazon Simple Storage Service (Amazon S3):
- **claims.csv**: una tabla con datos de reclamaciones sin procesar.
- **customers.csv**: una tabla con datos de clientes sin procesar.
- **claims_preprocessed.csv**: una tabla con datos de reclamaciones procesados.
- **customers_preprocessed.csv**: una tabla con datos de clientes procesados.
- **claims_customer.csv**: una tabla unida en **policy_id** de los datos procesados de reclamaciones y clientes.

Para este laboratorio, comience con las tablas **claims.csv** y **customers.csv**. Procéselas en el **Desafío 1** con Amazon SageMaker Data Wrangler. Si se traba o quiere una referencia de lo que debería ser el conjunto de datos procesado, puede revisar las tablas preprocesadas.

Para este conjunto de datos, el objetivo es **fraud**, una columna en la tabla de reclamaciones.

La tabla de reclamaciones incluye los siguientes campos: 

- **policy_id**: el identificador único de la póliza.
- **driver_relationship**: una lista de relaciones (Cónyuge, Uno mismo, Hijo, Otro, N/D).
- **incident_type**: el tipo de incidente informado (Entrada forzada, Choque, Robo).
- **collision_type**: la ubicación del choque (Frontal, Trasero, Lateral, N/D).
- **incident_severity**: la gravedad del incidente (Menor, Mayor, Total).
- **authorities_contacted**: el tipo de autoridades que se contactó primero (Ninguna, Policía, Ambulancia, Bomberos).
- **num_vehicles_involved**: el número de vehículos involucrados en el incidente (de 1 a 6).
- **num_injuries**: el número de lesionados en el incidente (de 1 a 4).
- **num_witnesses**: el número de testigos del incidente (de 1 a 5).
- **police_report_available**: si hay una denuncia a la policía disponible o no (sí o no).
- **injury_claim**: el valor de la reclamación por las lesiones en dólares estadounidenses (300 a 576 300 USD).
- **vehicle_claim**: el valor de la reclamación por los daños al vehículo en dólares estadounidenses (1000 a 51 051 USD).
- **total_claim_amount**: el valor total reclamado por lesiones y daños (2100 a 588 868 USD).
- **incident_month**: el mes del incidente (del 1 al 12).
- **incident_day**: el día del incidente (del 1 al 31).
- **incident_dow**: el día de la semana del incidente (del 0 al 6, que representan del domingo al sábado).
- **incident_hour**: la hora del incidente (del 0 al 23).
- **fraud**: si la póliza era fraudulenta o no (0 o 1).

La tabla de clientes incluye los siguientes campos:

- **policy_id**: el identificador único de la póliza.
- **customer_age**: la edad del cliente (de 18 a 70).
- **months_as_customer**: el número de meses que el cliente ha pagado el seguro (de 1 a 495).
- **num_claims_past_year**: el número de reclamaciones que el cliente realizó en el último año.
- **num_insurers_past_5_years**: el número de aseguradoras que el cliente tuvo en los últimos 5 años.
- **policy_state**: el estado en el que vive el cliente (AZ, CA, ID, NV, OR, WA).
- **policy_deductable**: el valor deducible de la póliza en dólares estadounidenses (de 750 a 1100 USD).
- **policy_annual_premium**: la prima anual de la póliza en dólares estadounidenses (de 2200 a 3000 USD).
- **policy_liability**: los importes máximos de responsabilidad por lesiones corporales, dividido en lesiones corporales simples y todas las lesiones corporales (15/30, 25/50, 60/90, 100/200).
- **customer_zip**: el código postal del cliente (de 83201 a 99362).
- **customer_gender**: el sexo del cliente (Masculino, Femenino, Otro, Desconocido).
- **customer_education**: el nivel de educación del cliente (Menos que escuela secundaria, Escuela secundaria, Tecnicatura, Licenciatura, Estudios superiores).
- **auto_year**: el año de fabricación del automóvil (de 2001 a 2020).

Puede realizar una unión interna de estas tablas en la columna **policy_id**.

## Navegación del laboratorio de desafíos

Este laboratorio está configurado con vínculos que navegan entre las tareas de los desafíos y el apéndice al final del cuaderno. Si desea revisar un elemento del apéndice, seleccione el hipervínculo asociado. Cuando desee regresar al desafío en el que está trabajando actualmente, seleccione el hipervínculo de la tarea correspondiente en el apéndice.

El laboratorio está organizado de la siguiente manera:

- Desafío 1: analizar y preparar el conjunto de datos con SageMaker Data Wrangler
- Desafío 2: crear grupos de funciones en SageMaker Feature Store
- Desafío 3: entrenar el modelo
- Desafío 4: evaluar el sesgo del modelo
- Desafío 5: transformación por lotes
- Desafío 6: crear una canalización automatizada
- Apéndice

## Desafío 1: analizar y preparar el conjunto de datos con SageMaker Data Wrangler

La Consultora CualquierCompañía ha recibido una solicitud para analizar conjuntos de datos sobre fraudes en seguros de automóviles y crear un modelo que ayude a predecir si es probable que las nuevas reclamaciones sean fraudulentas o no. Tiene 5000 registros de clientes en los que ha etiquetado cada reclamación como fraudulenta o no. Puede utilizar estos datos para entrenar, probar y validar su modelo antes de ejecutar una interferencia en una nueva colección de registros por lotes.

Utilice las funciones de análisis de Amazon SageMaker Data Wrangler para visualizar las distribuciones de datos en las columnas importantes y verificar la correlación entre las columnas y la filtración de objetivos. Luego, cree un modelo de referencia rápido. A continuación, utilice las funciones de procesamiento de SageMaker Data Wrangler para transformar las columnas de modo que sean más adecuadas para entrenar un modelo de mayor rendimiento. 

Para completar esta tarea, realice las siguientes subtareas:

- Revise los datos.
- Realice un análisis de datos exploratorio en Amazon SageMaker Studio.
- Utilice un trabajo de procesador de Amazon SageMaker Clarify para ejecutar un informe de sesgos.
<b>- </b>Prepare los datos.

El tiempo estimado para completar este desafío es de *100* minutos.

### Tarea 1.1: revisar los datos

<a id="task1-1-continue"></a>

Acceda al conjunto de datos tabulares de seguro de automóviles almacenado en su repositorio y revise una muestra del conjunto de datos. El repositorio contiene dos tablas sin procesar. Una de los datos de clientes llamada **customers.csv** y otra de los datos de reclamaciones llamada **claims.csv**.

**Sugerencia 1**: las tablas sin procesar se encuentran en la carpeta **./data/**.

**Sugerencia 2**: las tablas **claims.csv** y **customers.csv** son las tablas sin procesar.

Tómese un momento para explorar las tablas. ¿Hay algún campo que se destaque? ¿Hay algún campo que requiera un procesamiento previo?

Para conocer los pasos detallados sobre cómo revisar los datos, consulte <a href="#task1-1" target="_self">**Revisar los datos (Tarea 1.1)**</a> en la sección *Apéndice*.

Una vez que haya accedido a la tabla de fraudes de seguros de automóviles y revisado una muestra del conjunto de datos, habrá completado esta tarea.

In [ ]:
#add_your_task_1_1_code_here



## Tarea 1.2: realizar un análisis de datos exploratorio en SageMaker Studio 

<a id="task1-2-continue"></a>

Revise los datos para realizar un análisis de datos exploratorio, identifique los posibles problemas en el conjunto de datos y verifique las fuertes correlaciones entre cualquier columna y el objetivo. Puede explorar los datos en SageMaker Data Wrangler y en el cuaderno.

Tómese un tiempo para revisar específicamente los siguientes elementos:
- **Column histograms (Histogramas de columnas)**: revise las columnas en formato visual y verifique qué tipos de valores hay en el conjunto de datos.
- **Quick model (Modelo rápido)**: revise el conjunto de datos y piense en un resultado esperado del modelo.
- **Feature correlation (Correlación de funciones)**: verifique si hay una fuerte correlación entre las columnas y el objetivo.
- **Target leakage (Filtración de objetivos)**: verifique si hay algún dato que dependa del valor del objetivo.

Al abrir SageMaker Data Wrangler se abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista en la ventana principal de SageMaker Studio con la pantalla dividida, puede arrastrar la pestaña **capstone.ipynb** hacia un lado o seleccionar (con el botón derecho) la pestaña **capstone.ipynb** y luego **New View for Notebook (Nueva vista para el cuaderno)**. Ahora puede tener las instrucciones visibles mientras trabaja con el flujo de SageMaker Data Wrangler
- **Opción 2**: cambiar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar los pasos de SageMaker Data Wrangler, seleccione la pestaña **capstone.ipynb** para volver al cuaderno.

Si cuando crea un nuevo archivo aparece el error **“The following instance type is not available: ml.m5.4xlarge. Try selecting a different instance below”** (El siguiente tipo de instancia no está disponible: ml.m5.4xlarge. Intente seleccionar un tipo de instancia diferente), puede seleccionar otro tipo de instancia. Luego intente con **ml.m5.8xlarge**.

Si aparece el mensaje **An error occurred loading this view** (Se produjo un error al cargar esta vista), cierre la pestaña **untitled.flow** y vuelva a abrir el archivo del flujo desde el navegador de archivos.

**Sugerencia 1**: hay muchas formas de explorar el conjunto de datos. Abra un flujo en SageMaker Data Wrangler para comenzar. Necesitará importar **claims.csv** y **customers.csv** a SageMaker Data Wrangler desde el bucket de S3 que contiene **databucket-** en el nombre.

**Sugerencia 2**: para importar una segunda tabla, regrese a su **Data flow** (Flujo de datos), luego seleccione la pestaña **Import** (Importar) para importar otro conjunto de datos.

**Sugerencia 3**: **Get data insights** (Obtener información de datos) y **Add analysis** (Agregar análisis) son dos maneras de explorar datos en SageMaker Data Wrangler. Después de revisar algunos gráficos de muestra de sus datos, puede utilizar otras herramientas de trazado en el cuaderno para analizar los datos si lo desea. Se han instalado las bibliotecas **plt** y **sns**. No dude en utilizar cualquier herramienta de análisis que conozca para explorar el conjunto de datos.  

**Sugerencia 4**: intente unir dos tablas con la función **Join** (Unir) en **policy_id**. A continuación, ejecute otro reporte de información. Puede utilizar la función de unión **Inner** (Interna) para estas tablas.

¿Obtuvo resultados más significativos con un conjunto de datos unidos?

Para conocer los pasos detallados sobre cómo explorar un conjunto de datos en SageMaker Studio, consulte <a href="#task1-2-1" target="_self">**Explorar un conjunto de datos en SageMaker Studio (Tarea 1.2)**</a> en la sección *Apéndice*.

Para conocer los pasos detallados sobre cómo explorar un conjunto de datos en el cuaderno, consulte <a href="#task1-2-2" target="_self">**Explorar un conjunto de datos en el cuaderno (Tarea 1.2)**</a> en la sección *Apéndice*.

Una vez que haya procesado sus datos con SageMaker Data Wrangler, explorado el conjunto de datos e identificado los pasos de procesamiento que quiere realizar, habrá completado esta tarea.

In [ ]:
#add_your_task_1_2_code_here



### Tarea 1.3: utilizar un trabajo de procesador de SageMaker Clarify para ejecutar un informe de sesgos

<a id="task1-3-continue"></a>

Utilice SageMaker Clarify para ejecutar un informe de sesgos previo al entrenamiento para detectar desequilibrios de clase en los datos. Utilice un flujo de SageMaker Data Wrangler para ejecutar el informe de sesgos en SageMaker Studio.
1. Comience por unir las dos tablas.

- Unir: realizar una unión **Interna** de **claims.csv** y **customers.csv** en **policy_id**.

**Sugerencia 1**: para crear un informe de sesgos previo al entrenamiento, agregar un nuevo análisis al flujo de SageMaker Data Wrangler y seleccionar **Bias Report** (Informe de sesgos) en **Analysis type** (Tipo de análisis).

**Sugerencia 2**: puede ejecutar el informe de sesgos varias veces y seleccionar diferentes funciones para analizar cada vez.

Para conocer los pasos detallados sobre cómo unir tablas con SageMaker Data Wrangler, consulte <a href="#task1-3-1" target="_self">**Unión de tablas en SageMaker Studio (Tarea 1.3)**</a> en la sección *Apéndice*.

Para conocer los pasos detallados sobre cómo ejecutar un informe de sesgos previo al entrenamiento, consulte <a href="#task1-3-2" target="_self">**Ejecutar un informe de sesgo previo al entrenamiento (Tarea 1.3)**</a> en la sección *Apéndice*.

Una vez que haya ejecutado el informe de sesgos previo al entrenamiento y revisado el informe, habrá completado esta tarea.

### Tarea 1.4: preparar los datos

<a id="task1-4-continue"></a>

Prepare su conjunto de datos con SageMaker Data Wrangler. Céntrese en las siguientes transformaciones, pero no dude en incluir otras:

- Codificación categórica (Codificación one-hot): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type** y **policy_state**.
- Codificación ordinal: **customer_education**, **policy_liability**, **incident_severity** y **police_report_available**.
- Analizar columna como tipo: **vehicle_claim** y **total_claim_amount** de **Float** (Flotante) a **Long** (Larga).
- Administrar columnas (Eliminar columna): **customer_zip**.
- Administrar columnas (Mover columna): **fraud** (usar **Move to start** [Mover para comenzar]).
- Administrar columnas (Cambiar nombre de columna): reemplazar el símbolo **/** de **collision_type_N/A** y **driver_relationship_N/A** con un **_**.
- Administrar columnas (Cambiar nombre de columna): cambiar el nombre **policy_id_0** a **policy_id**.

**Sugerencia 1**: una la tabla **claims** a las tablas **customers** con una unión en SageMaker Data Wrangler. 

**Sugerencia 2**: una las dos tablas en la columna **policy_id**.

**Sugerencia 3**: agregue las transformaciones con la opción **Add transform (Agregar transformación)**.

¿Qué transformaciones cree que afectan más al entrenamiento de modelos?

Para conocer los pasos detallados sobre cómo preparar los datos con SageMaker Data Wrangler, consulte <a href="#task1-4-1" target="_self">**Preparar datos en SageMaker Data Wrangler (Tarea 1.4)**</a> en la sección *Apéndice*.

Si desea importar un conjunto de datos procesados de ejemplo, consulte <a href="#task1-4-2" target="_self">**Importar un conjunto de datos procesados de ejemplo (Tarea 1.4)**</a> en la sección *Apéndice*. 


## Desafío 2: crear grupos de funciones en SageMaker Feature Store

Ahora que ya ha procesado el conjunto de datos, cree funciones y grupos de funciones para utilizar en análisis futuros. Utilice SageMaker Feature Store para almacenarlas en un grupo de funciones y consultarlas cuando entrene su modelo.

Para completar esta tarea, realice todas las subtareas siguientes:

1. Exportar funciones al almacén de funciones de SageMaker.
2. Consultar el grupo de funciones en un almacén sin conexión con Amazon Athena.

El tiempo estimado para completar este desafío es de *30* minutos.

### Tarea 2.1: exportar funciones al almacén de funciones de SageMaker

<a id="task2-1-continue"></a>

Utilice la función **Export to** (Exportar a) de SageMaker Data Wrangler para crear un cuaderno de Jupyter personalizado. El cuaderno crea una definición de función y un grupo de funciones. El cuaderno ingiere los registros en el grupo de funciones. En el cuaderno, complete los siguientes pasos:

- Establezca los valores de record (registro) y event_time (hora_evento).
- Ejecute las celdas del cuaderno para crear un grupo de funciones.
- Ejecute las celdas del cuaderno para confirmar el grupo de funciones creado.
- Ejecute las celdas del cuaderno para ingerir los registros en el grupo de funciones.

**Sugerencia 1**: todos estos pasos se pueden completar en SageMaker Studio. Cuando haya finalizado de crear su grupo de funciones, puede regresar a este cuaderno para continuar con la Tarea 2.2.

**Sugerencia 2**: agregue una transformación personalizada para crear la columna **event_time**. 

**Sugerencia 3**: el código para agregar la transformación personalizada es el siguiente:
    


In [ ]:
import time
import datetime
import pandas as pd
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


**Sugerencia 4**: al final del flujo de SageMaker Data Wrangler, seleccione el ícono **+**, luego la opción **Export to** (Exportar a) y finalmente **SageMaker Feature Store** (Almacén de funciones de SageMaker) (a través del cuaderno de Jupyter).

**Sugerencia 5**: para desconectar el almacén en línea puede cambiar el valor **enable_online_store** de **True** a **False**.

¿Cómo utilizaría SageMaker Feature Store para almacenar y consultar sus registros para el entrenamiento, a diferencia de la inferencia?

Para conocer los pasos detallados sobre cómo crear un grupo de funciones con la opción **Export to** (Exportar a), consulte <a href="#task2-1" target="_self">**Crear un grupo de funciones con la opción Export to (Tarea 2.1)**</a> en la sección *Apéndice*. 

Una vez que haya creado un grupo de funciones e ingerido los datos en él, habrá completado esta tarea.

## Tarea 2.2: consultar el grupo de funciones en un almacén sin conexión con Athena

<a id="task2-2-continue"></a>

Utilice Athena para extraer registros de un almacén de datos sin conexión. En el próximo desafío puede dividir estos registros en conjuntos de entrenamiento, prueba y validación.

Utilice la celda de código proporcionada a continuación para hacer las llamadas a la API de Amazon Athena. Puede utilizar la consola de Amazon Athena para hacer la consulta, pero eso queda fuera del alcance de este laboratorio.

**Sugerencia 1**: puede crear una consulta en Athena con **feature_group.athena_query()** y obtener el nombre de la tabla con **query.table_name**.

**Sugerencia 2**: puede ejecutar una consulta con **query.run(query_string=query_string, output_location=output_location)** y leer el valor obtenido como un marco de datos con **query.as_dataframe()**.

¿Cómo podría utilizar **event_time** para hacer un seguimiento de las funciones que se producen en diferentes puntos de la línea de tiempo de su conjunto de datos?

Para conocer los pasos detallados sobre cómo extraer registros de un almacén de datos sin conexión con Athena, consulte <a href="#task2-2" target="_self">**Extraer registros de un almacén de datos sin conexión con Athena (Tarea 2.2)**</a> en la sección *Apéndice*.

Una vez que haya guardado la consulta de Athena como una variable de marco de datos, habrá completado esta tarea.

In [ ]:
#add_your_task_2_2_code_here



## Desafío 3: entrenar el modelo

Su modelo está listo para el entrenamiento. Divida los datos en conjuntos de datos de entrenamiento, prueba y validación para entrenar al modelo. 

SageMaker Autopilot se ejecutó en estos datos antes y obtuvo una **F1** de **0,616**, una **exactitud** de **0,978**, una **AUC** de **0,918** y una **exhaustividad** de **0,539**. Para conocer más sobre estás métricas que produce SageMaker Autopilot, consulte el documento *piloto-automático-métricas-validación* en la sección de *Recursos adicionales* para obtener más información.

Mientras realiza el entrenamiento y el ajuste, trabaje para alcanzar o superar las puntuaciones de SageMaker Autopilot y confirmar que el depurador de Amazon SageMaker no informe errores.

Para completar esta tarea, realice las siguientes subtareas:

- Cree un experimento y una ejecución.
- Divida los datos en conjuntos de datos de entrenamiento, prueba y validación.
- Configure y ejecute un trabajo de entrenamiento.
    - Ejecute un trabajo de entrenamiento básico.
    - Ejecute un trabajo de entrenamiento básico con el depurador de SageMaker habilitado y analice los informes (opcional).
- Realice ajustes de hiperparámetros.

El tiempo estimado para completar este desafío es de *110* minutos.


### Tarea 3.1: dar nombre a un experimento y una ejecución

<a id="task3-1-continue"></a>

Establezca las variables para dar nombre al experimento y a las ejecuciones. 
Un experimento requiere un **experiment_name**, un **run_name** y una **description**. 

Para conocer los pasos detallados sobre cómo crear las variables, consulte <a href="#task3-1" target="_self">**Dar nombre a un experimento y una ejecución (Tarea 3.1)**</a> en la sección *Apéndice*.


In [ ]:
#add_your_task_3_1_code_here



### Tarea 3.2: dividir los datos en conjuntos de datos de entrenamiento, prueba y validación

<a id="task3-2-continue"></a>

Utilice las funciones de la consulta en SageMaker Feature Store y divídalas en conjuntos de datos de entrenamiento, prueba y validación.

**Sugerencia 1**: utilice **np.split** para dividir los conjuntos de datos en tres particiones.

**Sugerencia 2**: utilice **to_csv** para crear archivos CSV y utilice **S3Uploader.upload** para agregar los archivos a Amazon S3.

**Sugerencia 3**: el producto final de la división debería ser una variable **data_inputs** con valores para **train** (entrenamiento) y **validation** (validación).

Para conocer los pasos detallados sobre cómo dividir los datos en conjuntos de datos de entrenamiento, prueba y validación, consulte <a href="#task3-2" target="_self">**Dividir datos en conjuntos de datos de entrenamiento, prueba y validación (Tarea 3.2)**</a> en la sección *Apéndice*. 

Una vez que haya dividido los datos en conjuntos de datos de entrenamiento, prueba y validación, habrá completado esta tarea.

In [ ]:
#add_your_task_3_2_code_here



### Tarea 3.3: configurar y ejecutar un trabajo de entrenamiento

<a id="task3-3-continue"></a>

Comience su primer trabajo de entrenamiento, configure el contenedor, un estimador y los hiperparámetros. A continuación, entrene el modelo con **fit()**. Si desea examinar informes más detallados, habilite el depurador de SageMaker con **DebuggerHookConfig**.

**Sugerencia 1**: utilice el contenedor **XGBoost** con la versión **1.5-1**.

**Sugerencia 2**: para comenzar, configure los hiperparámetros **eta**, **gamma**, **max_depth**, **min_child_weight**, **num_round**, **objective** y **subsample**.

**Sugerencia 3**: utilice **data_inputs** que ha creado antes en el Desafío 1 como valor **inputs** para su trabajo de entrenamiento.

**Sugerencia 4**: para configurar un trabajo de entrenamiento puede ajustar inputs y experiment_config. Experiment_config debería contener una **sagemaker_session**, un **run_name** y un **experiment_name**.

**Sugerencia 5**: si desea utiliza el depurador de SageMaker, configure el **DebuggerHookConfig**, el **ProfilerConfig** y el objeto del depurador **rule**.

¿Qué hiperparámetros tienen más probabilidades de causar un mayor impacto en el rendimiento y la exactitud de su modelo? ¿Qué hiperparámetros piensa ajustar primero?

Para conocer los pasos detallados sobre cómo configurar y ejecutar un trabajo de entrenamiento básico, consulte <a href="#task3-3-1" target="_self">**Configurar y ejecutar un trabajo de entrenamiento básico (Tarea 3.3)**</a> en la sección *Apéndice*.

Para conocer los pasos detallados sobre cómo configurar y ejecutar un trabajo de entrenamiento básico con el depurador habilitado y analizar los informes, consulte <a href="#task3-3-2" target="_self">**Configurar y ejecutar un trabajo de entrenamiento básico con el depurador de SageMaker habilitado y analizar los informes (Tarea 3.3)**</a> en la sección *Apéndice*.

Una vez que haya finalizado uno o más trabajos de entrenamiento, habrá completado esta tarea.

In [ ]:
#add_your_task_3_3_code_here



## Tarea 3.4: ajuste de hiperparámetros

<a id="task3-4-continue"></a>

Ahora que ya ha finalizado un trabajo de entrenamiento y lo ha analizado, ajuste los rangos de los hiperparámetros en función de sus hallazgos y ejecute más trabajos de entrenamiento para mejorar el modelo.

**Sugerencia 1**: para comenzar, establezca los rangos de los hiperparámetros **alpha**, **eta**, **max_depth**, **min_child_weight** y **num_round**.

**Sugerencia 2**: cuando ejecute **HyperparameterTuner**, asegúrese de configurar **objective_metric_name** y **objective_type** en función de sus hallazgos.

**Sugerencia 3**: para verificar que ha mejorado los resultados de SageMaker Autopilot, abra el menú **Experiments and runs** (Experimentos y ejecuciones) en **SageMaker resources** (Recursos de SageMaker). Dentro de su ejecución, vea las **Metrics** (Métricas). **ObjectiveMetric** (Métrica objetivo) debería ser mayor que la puntuación de **F1** de **0,616** y **validation:auc** debería tener un **Final value** (Valor final) mayor a la puntuación de SageMaker Autopilot de **0,918**.

Cuando ajustó los hiperparámetros, ¿cuál fue el que más mejoró el rendimiento del modelo?

Para conocer los pasos detallados sobre cómo configurar los rangos de los hiperparámetros de entrenamiento, consulte <a href="#task3-4" target="_self">**Configurar los rangos de los hiperparámetros de entrenamiento (Tarea 3.4)**</a> en la sección *Apéndice*.

Una vez que haya configurado los rangos de los hiperparámetros de entrenamiento y haya comenzado más trabajos de entrenamiento, habrá completado esta tarea.

In [ ]:
#add_your_task_3_4_code_here



## Desafío 4: evaluar el sesgo del modelo

Ahora que ha entrenado el modelo, evalúelo con Amazon SageMaker Clarify. Si encuentra algún problema, puede eliminar el desequilibrio detectado y volver a entrenar el modelo.

Para completar esta tarea, realice las siguientes subtareas:

- Cree un modelo a partir del trabajo de entrenamiento.
- Cree una configuración del modelo de SageMaker Clarify.
- Cree una configuración de sesgos de SageMaker Clarify.
- Utilice un trabajo de procesador de SageMaker Clarify para ejecutar informes de sesgos, datos y modelos.
- Elimine el desequilibrio detectado con SageMaker Clarify (opcional).
- Vuelva a entrenar el modelo (opcional).

El tiempo estimado para completar este desafío es de *80* minutos.

## Tarea 4.1: crear un modelo a partir del trabajo de entrenamiento

<a id="task4-1-continue"></a>

Cree un modelo XGBoost, llamando **create_model** con el **model_name**, **role** y **container_def** que usted defina.

**Sugerencia 1**: llame a **xgb.create_model()** y seleccione un nombre para su modelo.

**Sugerencia 2**: utilice su sesión y llame a **create_model**, indicando el **model_name**, **role** y **container_def**.

Para conocer los pasos detallados sobre cómo crear un modelo, consulte <a href="#task4-1" target="_self">**Crear un modelo (Tarea 4.1)**</a> en la sección *Apéndice*.

Una vez que haya creado un modelo, habrá completado esta tarea.

In [ ]:
#add_your_task_4_1_code_here



### Tarea 4.2: crear una configuración del modelo de SageMaker Clarify

<a id="task4-2-continue"></a>

Cree una configuración del modelo de SageMaker Clarify con **SageMakerClarifyProcessor**.

**Sugerencia 1**: configure **instance_count** e **instance_type**.

**Sugerencia 2**: utilice el **rol** y la **sesión** creados al comienzo del laboratorio del curso de culminación.

Para conocer los pasos detallados sobre cómo crear una configuración del modelo de SageMaker Clarify, consulte <a href="#task4-2" target="_self">**Crear una configuración del modelo de SageMaker Clarify (Tarea 4.2)**</a> en la sección *Apéndice*.

Una vez que haya creado una configuración del modelo de SageMaker Clarify, habrá completado esta tarea.

In [ ]:
#add_your_task_4_2_code_here



### Tarea 4.3: crear una configuración de sesgos de SageMaker Clarify

<a id="task4-3-continue"></a>

Cree una configuración de datos, una configuración del modelo, una configuración de etiquetas y una configuración de sesgos.

**Sugerencia 1**: comience con **DataConfig** y configure la ruta de entrada, la ruta de salida, los encabezados y el tipo de conjunto de datos.

**Sugerencia 2**: a continuación, cree un **ModelConfig** y seleccione el tipo de contenido y de aceptación, el nombre del modelo, el tipo de instancia y el recuento de instancias.

**Sugerencia 3**: luego, cree un **ModelPredictedLabelConfig** y configure el umbral de probabilidad.

**Sugerencia 4**: finalmente, cree un **BiasConfig** y configure el valor de la etiqueta o umbral, el nombre de la faceta y los valores de las facetas o umbrales.

¿Qué facetas desea explorar primero en su informe de sesgos? ¿Existen funciones especialmente susceptibles a los sesgos?

Para conocer los pasos detallados sobre cómo crear una configuración de sesgos de SageMaker Clarify, consulte <a href="#task4-3" target="_self">**Crear una configuración de sesgos de SageMaker Clarify (Tarea 4.3)**</a> en la sección *Apéndice*.

Una vez que haya creado la configuración de sesgos de SageMaker Clarify, habrá completado esta tarea.

In [ ]:
#add_your_task_4_3_code_here



### Tarea 4.4: utilizar un trabajo de procesador de SageMaker Clarify para ejecutar informes de sesgos, datos y modelos

<a id="task4-4-continue"></a>

Ya ha elegido todas sus configuraciones para los informes de sesgos, datos y modelos. Ahora, ejecute los informes.

**Sugerencia 1**: introduzca los valores de **data_config**, **bias_config**, **model_predicted_label_config** y **model_config** para **run_bias**.

**Sugerencia 2**: necesita configurar **pre_training_methods** y **post_training_methods**.

Para conocer los pasos detallados sobre cómo ejecutar informes de sesgos, datos y modelos con SageMaker Clarify, consulte <a href="#task4-4" target="_self">*Ejecutar informes de sesgos, datos y modelos con SageMaker Clarify (Tarea 4.4)*</a> en la sección *Apéndice*.

Una vez que haya utilizado un trabajo de procesador de SageMaker Clarify para ejecutar informes de sesgos, datos y modelos, habrá completado esta tarea.

In [ ]:
#add_your_task_4_4_code_here



### Tarea 4.5: eliminar el desequilibrio detectado con SageMaker Clarify (opcional)

<a id="task4-5-continue"></a>

Existen muchas formas de eliminar el desequilibrio detectado con SageMaker Clarify. Utilice cualquier método que conozca. En este laboratorio, se proporciona un ejemplo de Técnica de sobremuestreo sintético de minorías (Synthetic Minority Over-sampling Technique, SMOTE) que elimina el sesgo de las columnas.

**Sugerencia 1**: cuando haya terminado de eliminar el desequilibrio y desee volver a realizar la prueba, cree un nuevo marco de datos con las nuevas muestras. En la siguiente tarea cargará un nuevo archivo CVS.

Para conocer los pasos detallados sobre cómo eliminar el desequilibrio, consulte <a href="#task4-5" target="_self">*Eliminar el desequilibrio (Tarea 4.5)**</a> en la sección *Apéndice*.

Una vez que haya eliminado cualquier desequilibrio detectado con SageMaker Clarify, habrá completado esta tarea.

In [ ]:
#add_your_task_4_5_code_here



### Tarea 4.6: volver a entrenar el modelo (opcional)

<a id="task4-6-continue"></a>

Cargue el nuevo archivo en Amazon S3. A continuación, cree un nuevo estimador y repita en entrenamiento con los nuevos datos.

**Sugerencia 1**: utilice **s3_client.upload_file** para cargar el nuevo archivo en el bucket.

**Sugerencia 2**: utilice **xgboost_starter_script.py** y llame a **XGBoost**. Luego, repita el entrenamiento con los nuevos datos.

¿El nuevo modelo entrenado obtuvo una puntuación F1 más alta? Si utilizó el depurador de SageMaker, ¿fue capaz de resolver todos los problemas detectados?

Para conocer los pasos detallados sobre cómo volver a entrenar el modelo, consulte <a href="#task4-6" target="_self">*Volver a entrenar el modelo (Tarea 4.6)*</a> en la sección *Apéndice*.

Una vez que haya vuelto a entrenar el modelo, habrá completado esta tarea.

In [ ]:
#add_your_task_4_6_code_here



## Desafío 5: transformación por lotes

Su modelo está listo para la implementación. Utilice un trabajo de transformación por lotes con registros por lotes y vea los datos de predicción y exactitud en Amazon S3. A continuación, realice una limpieza de algunas de sus instancias de SageMaker.

Para completar esta tarea, realice las siguientes subtareas:

- Cree un trabajo de transformación por lotes para su modelo.
- Vea los datos de predicción en Amazon S3.
- Realice una limpieza de las instancias de SageMaker (opcional).

El tiempo estimado para completar este desafío es de *40* minutos.

### Tarea 5.1: crear un trabajo de transformación por lotes para su modelo

<a id="task5-1-continue"></a>

Cree un trabajo de transformación por lotes mediante el uso de **transformer** en el estimador del modelo. A continuación, ejecute el trabajo por lotes.

**Sugerencia 1**: utilice el transformador y configure **instance_count**, **instance_type**, **strategy**, **assemble_with** y **output_path**.

**Sugerencia 2**: envíe los datos de la prueba, que se encuentran en **test_path** al punto de enlace y aguarde los resultados.

Para conocer los pasos detallados sobre cómo crear un trabajo de transformación por lotes, consulte <a href="#task5-1" target="_self">*Crear un trabajo de transformación por lotes (Tarea 5.1)*</a> en la sección *Apéndice*.

Una vez que haya creado un trabajo de transformación por lotes y lo haya ejecutado con un conjunto de registros, habrá completado esta tarea.

In [ ]:
#add_your_task_5_1_code_here



### Tarea 5.2: ver los datos de predicción en Amazon S3

<a id="task5-2-continue"></a>

Cuando su trabajo de transformación por lotes esté completo, lea los datos en Amazon S3. 

**Sugerencia 1**: puede copiar los datos de la salida del transformador mediante el uso de **%aws s3 cp --recursive $transformer.output_path ./**

**Sugerencia 2**: cuando tenga los datos, puede visualizarlos con **%head test_data_batch.csv.out**

Eche un vistazo a las predicciones. ¿Hay alguna predicción que sorprenda?

Para conocer los pasos detallados sobre cómo ver los datos de predicción de un trabajo de transformación por lotes, consulte <a href="#task5-2" target="_self">*Ver los datos de predicción de un trabajo de transformación por lotes (Tarea 5.2)*</a> en la sección *Apéndice*.

Una vez que haya visualizado los datos de la predicción del trabajo de transformación por lotes, habrá completado esta tarea.

In [ ]:
#add_your_task_5_2_code_here



### Tarea 5.3: realizar una limpieza de las instancias de SageMaker (opcional)

Para mantener los costos bajos, la práctica recomendada es eliminar las instancias que ya no utiliza. Puede eliminar rápidamente las instancias con SageMaker Studio. Tómese un momento para abrir su lista de recursos actuales en SageMaker Studio y cierre las instancias restantes.

Si planea completar la siguiente tarea de canalización, **deje la instancia del cuaderno en ejecución**.

**Sugerencia 1**: para ver una lista de las instancias en ejecución, seleccione el ícono **Running Terminals and Kernels** (Terminales y kernels en ejecución) en SageMaker Studio.

**Sugerencia 2**: puede utilizar el ícono **Shut down** (Cerrar) para detener una instancia.

<a id="task5-3-continue"></a>

Para conocer los pasos detallados sobre cómo realizar una limpieza de las instancias de SageMaker en SageMaker Studio, consulte <a href="#task5-3" target="_self">** Realizar una limpieza de las instancias de SageMaker en SageMaker Studio (Tarea 5.3)**</a> en la sección *Apéndice*.

Una vez que haya detenido todas las instancias de SageMaker en SageMaker Studio, habrá completado esta tarea.

## Desafío 6: crear una canalización automatizada (opcional)

Ahora que ya ha utilizado el SDK para Python de Amazon SageMaker y Amazon SageMaker Studio para el flujo de trabajo de machine learning (ML), utilice las canalizaciones de SageMaker para escalar su flujo de trabajo. Proceda a través del script de canalización proporcionado en el entorno de laboratorio para completar este desafío. 

- Cree los pasos de la canalización.
    - Haga una consulta de los datos procesados desde SageMaker Feature Store.
    - Entrene y ajuste el modelo.
    - Evalúe el modelo entrenado.
    - Realice un trabajo de transformación por lotes.
    - Registre un modelo.
    - Evalúe el entrenamiento del modelo con SageMaker Clarify.
- Defina y comience una canalización.
- Vea el seguimiento de linaje de ML.

El tiempo estimado para completar este desafío es de *120* minutos.


### Tarea 6.1: configurar una canalización

<a id="task6-1-continue"></a>

Utilice una plantilla de canalización y configure sus entradas y salidas. Cuando las configuraciones estén listas, ejecute la canalización. Su canalización puede incluir una amplia gama de pasos. A continuación hay una lista sugerida de los pasos para la configuración:
- **AutoModelProcess**: un paso de **Procesamiento** que extrae el archivo .csv y lo divide en conjuntos de datos de entrenamiento, prueba y validación.
- **AutoHyperParameterTuning**: un paso de **Ajuste** que utiliza una amplia gama de hiperparámetros y ajusta el modelo.
- **AutoEvalBestModel**: un paso de **Procesamiento** que crea un informe de evaluación para describir al mejor modelo.
- **CheckAUCScoreAutoEvaluation**: un paso de **Condición** que evalúa los modelos basados en una métrica de evaluación. 
- **AutoCreateModel**: un paso de **Modelo** que crea el modelo.
- **RegisterAutoModel-RegisterModel**: un paso de **Registro de modelo** que registra el modelo.
- **AutoModelConfigFile**: un paso de **Procesamiento** que crea un informe de sesgos.
- **AutoTransform**: un paso de **Transformación** que ejecuta un trabajo de transformación por lotes.
- **ClarifyProcessingStep**: un paso de **Procesamiento** que ejecuta un trabajo de SageMaker Clarify.

**Sugerencia 1**: hay muchos pasos de canalización para escoger. Para conocer más sobre los pasos de canalización y ver un código de muestra para cada paso, consulte el documento *Crear y administrar los pasos de canalización* en la sección de *Recursos adicionales* para obtener más información.

**Sugerencia 2**: comience un paso de **Procesamiento** para introducir sus datos. A continuación, cree un paso de **Ajuste** para ajustar el modelo. Luego, cree un paso de **Modelo** para crear su modelo.

**Sugerencia 3**: los pasos detallados contienen una solución de muestra e incluyen los pasos para crear informes de evaluación, un informe de sesgos, ejecutar un trabajo de transformación por lotes y ejecutar un trabajo en SageMaker Clarify. 

Para conocer los pasos detallados sobre cómo configurar una canalización, consulte <a href="#task6-1" target="_self">*Configurar una canalización (Tarea 6.1)*</a> en la sección *Apéndice*.

Una vez que haya configurado la canalización e iniciado un trabajo de canalización, habrá completado esta tarea.

In [ ]:
#add_your_task_6_1_code_here



### Tarea 6.2: supervisar la canalización

<a id="task6-2-continue"></a>

Supervise la canalización mientras está en ejecución y revise las entradas y las salidas. 

**Sugerencia 1**: utilice `RunPipeline.describe()` para describir la canalización que ha creado.

**Sugerencia 2**: puede ver los pasos de la canalización en ejecución en la UI de SageMaker Studio. Abra el menú **SageMaker resources** (Recursos de SageMaker), seleccione **Pipelines** (Canalizaciones) y seleccione la canalización que ha creado. 

Para conocer los pasos detallados sobre cómo supervisar una canalización, consulte <a href="#task6-2" target="_self">*Supervisar una canalización*</a> en la sección *Apéndice*.

Una vez que haya terminado de supervisar la canalización, habrá completado esta tarea.

In [ ]:
#add_your_task_6_2_code_here



¡Felicitaciones! Ha utilizado un conjunto de datos de seguros de automóviles para detectar reclamaciones posiblemente fraudulentas. Exploró una solución técnica para predecir la probabilidad de que una determinada reclamación de seguro de automóvil sea fraudulenta a través de SageMaker Studio y el SDK para Python de Amazon SageMaker.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, haga lo siguiente:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.

### Recursos adicionales

- [Autopilot metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)
- [Processing step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

## Apéndice

<a name="task1-1" id="task1-1"></a>

### Apéndice: revisar los datos (Tarea 1.1)

Para revisar sus datos, especifique la ruta y cargue los datos con Pandas. Tómese un momento para revisar una muestra de ambas tablas.

In [ ]:
#read-csv-files
claims_data = pd.read_csv("./data/claims_preprocessed.csv", index_col=0)
customers_data = pd.read_csv("./data/customers_preprocessed.csv", index_col=0)

In [ ]:
#claims-data-sample
claims_data.head()

In [ ]:
#customers-data-sample
customers_data.head()

Para continuar con este laboratorio, regrese a la <a href="#task1-1-continue" target="_self">Tarea 1.1</a>.

<a name="task1-2-1" id="task1-2-1"></a>

### Apéndice: explorar un conjunto de datos en SageMaker Studio (Tarea 1.2)

Comience su exploración de datos en SageMaker Data Wrangler. Importe sus archivos desde su bucket de S3 y analice los datos.

El siguiente paso lo llevará a una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una pantalla con la vista dividida en la ventana principal de SageMaker Studio, puede arrastrar la pestaña **capstone.ipynb** hacia un lado o elegir la pestaña **capstone.ipynb** y luego, desde la barra de herramientas, seleccionar **File** (Archivo) y **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones visibles a medida que explora el grupo de funciones.
- **Opción 2**: cambiar entre las pestañas de SageMaker Studio para seguir estas instrucciones.

1. En el lado izquierdo de SageMaker Studio, seleccione el ícono **Home** (Inicio).
1. Expanda la sección **Data** (Datos) y luego seleccione **Data Wrangler**.

SageMaker Studio abrirá la pestaña **Data Wrangler**.

1. Seleccione **+** y **Create Data Wrangler flow (Crear un flujo de Data Wrangler)**.

SageMaker Studio abrirá la pestaña **untitled.flow**.

1. Espere a que la pestaña **untitled.flow** termine de cargar, lo que se indica con una barra de progreso. Puede tomar de 2 a 3 minutos.

SageMaker Studio abrirá la página **Create connection** (Crear una conexión) en la pestaña *Data Wrangler*.

1. Abra el menú contextual (botón derecho del mouse) en la pestaña del archivo **untitled.flow** y, a continuación, seleccione **Rename Data Wrangler Flow...** (Cambiar el nombre del flujo de Data Wrangler...) para cambiar el nombre del archivo.

SageMaker Studio abrirá la ventana de mensaje **Rename File** (Cambiar nombre de archivo).

1. En **New Name** (Nombre nuevo), ingrese `CapstoneDataWrangler.flow`.
1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Rename</span> (Cambiar nombre).

La ventana de mensaje **Rename File** (Cambiar nombre de archivo) se cerrará.

1. En la pestaña **CapstoneDataWrangler.flow**, en la sección **Data sources** (Orígenes de datos), seleccione **Amazon S3**.

SageMaker Studio abrirá la página **Import a dataset from S3** (Importar un conjunto de datos de S3) en la pestaña *DataWrangler.flow*.

1. En la lista de buckets, abra el bucket que contiene **databucket** en el nombre.
1. Seleccione el primer conjunto de datos, un archivo llamado **claims.cvs**.

1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span> (Importar).

1. Regrese a la vista **Data flow** (Flujo de datos), seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.
1. Seleccione la pestaña **Import** (Importar), ubicada en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.

SageMaker Studio abrirá la página **Create connection** (Crear una conexión).

1. En la pestaña **CapstoneDataWrangler.flow**, en la sección **Data sources** (Orígenes de datos), seleccione **Amazon S3**.

SageMaker Studio abrirá la página **Import a dataset from S3** (Importar un conjunto de datos de S3) en la pestaña *DataWrangler.flow*.

1. En la lista de buckets, abra el bucket que contiene **databucket** en el nombre.
1. Seleccione el segundo conjunto de datos, un archivo llamado **customers.csv**.
1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span> (Importar).

1. Regrese a la vista **Data flow** (Flujo de datos), seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.

1. En la pestaña de flujo de datos, seleccione el signo **+** que se encuentra junto al ícono **Data types** (Tipos de datos) y seleccione **Get data insights** (Obtener información de datos).

1. Cree el informe y explore la información.

1. En la pestaña de flujo de datos, seleccione el signo **+** que se encuentra junto al ícono **Data types** (Tipos de datos) y seleccione **Add analysis** (Agregar análisis).

1. Cree el análisis y explore los resultados.

Utilice cualquier tipo de informe que lo ayude a explorar a fondo los conjuntos de datos. Una vez que haya finalizado, puede continuar con la siguiente tarea.

Para continuar con este laboratorio, regrese a la <a href="#task1-2-continue" target="_self">Tarea 1.2</a>.


<a name="task1-2-2" id="task1-2-2"></a>

### Apéndice: explorar un conjunto de datos en el cuaderno (Tarea 1.2)

Existen muchas formas en las que puede explorar los conjuntos de datos. A continuación hay varios ejemplos de pasos para la exploración de datos que puede utilizar. Utilícelos como referencia para comenzar a explorar los aspectos de los conjuntos de datos.

In [ ]:
#gender-graph
import matplotlib.pyplot as plt
customers_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

In [ ]:
#fraud-graph
claims_data.fraud.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Not Fraud", "Fraud"]);

In [ ]:
#education-category-graphs
educ = customers_data.customer_education.value_counts(normalize=True, sort=False)
plt.bar(educ.index, educ.values)
plt.xlabel("Customer Education Level");

In [ ]:
#claim-amount-graph
plt.hist(claims_data.total_claim_amount, bins=30)
plt.xlabel("Total Claim Amount")

In [ ]:
#claims-filed-graph
customers_data.num_claims_past_year.hist(density=True)
plt.suptitle("Number of Claims in the Past Year")
plt.xlabel("Number of claims per year")

In [ ]:
#paid-plot-graphs
sns.pairplot(
    data=customers_data, vars=["num_insurers_past_5_years", "months_as_customer", "customer_age"]
);

In [ ]:
#fraud-insurers-graph
combined_data = customers_data.join(claims_data)
sns.lineplot(x="num_insurers_past_5_years", y="fraud", data=combined_data);

In [ ]:
#months-as-customer-graph
sns.boxplot(x=customers_data["months_as_customer"]);

In [ ]:
#customer-age-graph
sns.boxplot(x=customers_data["customer_age"]);

In [ ]:
#fraud-gender-graph
combined_data.groupby("customer_gender_female").mean()["fraud"].plot.bar()
plt.xticks([0, 1], ["Male", "Female"])
plt.suptitle("Fraud by Gender");

In [ ]:
#correlation-matrix-graph
cols = [
    "fraud",
    "customer_gender_male",
    "customer_gender_female",
    "months_as_customer",
    "num_insurers_past_5_years",
]
corr = combined_data[cols].corr()

# plot the correlation matrix
sns.heatmap(corr, annot=True, cmap="Reds");

In [ ]:
#load-combined-data
combined_data = pd.read_csv("./data/claims_customer.csv")

In [ ]:
#remove-unnecessary-columns
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains("^Unnamed: 0")]
combined_data.head()

In [ ]:
#describe-combined-data
combined_data.describe()

In [ ]:
#generate-statistics
combined_stats = []

for col in combined_data.columns:
    combined_stats.append(
        (
            col,
            combined_data[col].nunique(),
            combined_data[col].isnull().sum() * 100 / combined_data.shape[0],
            combined_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            combined_data[col].dtype,
        )
    )

stats_df = pd.DataFrame(
    combined_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)
stats_df.sort_values("percent_largest_category", ascending=False)

In [ ]:
#heatmap-graph
sns.set_style("white")

corr_list = [
    "customer_age",
    "months_as_customer",
    "total_claim_amount",
    "injury_claim",
    "vehicle_claim",
    "incident_severity",
    "fraud",
]

corr_df = combined_data[corr_list]
corr = round(corr_df.corr(), 2)

fix, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, mask=mask, ax=ax, annot=True, cmap="OrRd")

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10, ha="right", rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10, va="center", rotation=0)

plt.show()

Para continuar con este laboratorio, regrese a la <a href="#task1-2-continue" target="_self">Tarea 1.2</a>.

<a name="task1-3-1" id="task1-3-1"></a>

### Apéndice: unión de tablas en SageMaker Studio (Tarea 1.3)

1. Regrese a la vista **Data flow** (Flujo de datos), seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.
1. Seleccione el símbolo **+** que se encuentra junto al ícono de **claims.CVS Data types** (Tipos de datos de claims.CVS) y, en el menú contextual, elija **Join** (Unir).

SageMaker Data Wrangler mostrará la página **Join** (Unir).

1. Seleccione el ícono **customers.csv Data types** (Tipos de datos customers.cvs).
1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span> (Configurar).
1. En **Join Type** (Tipo de unión), seleccione **Inner** (Interna).
1. En la sección **Columns** (Columnas):

    - En **Left** (Izquierda), seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.
    
    - En **Right** (Derecha), seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.

1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span> (Previsualizar).

1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span> (Agregar).

Para continuar con este laboratorio, regrese a la <a href="#task1-3-continue" target="_self">Tarea 1.3</a>.

<a name="task1-3-2" id="task1-3-2"></a>

### Apéndice: ejecutar un informe de sesgos previo al entrenamiento (Tarea 1.3)

Cree un informe de sesgos de SageMaker Clarify con el flujo de SageMaker Data Wrangler.

1. Seleccione la pestaña **CapstoneDataWrangler.flow**.
1. Vaya a la vista **Data flow** (Flujo de datos). Si es necesario, seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña **DataWranglerLab.flow**. 
1. Seleccione el símbolo **+** que se encuentra junto al ícono **Join** (Unir) y, en el menú contextual, seleccione **Add analysis** (Agregar análisis).
1. En la sección *Create analysis** (Crear análisis):

- En **Analysis type** (Tipo de análisis), seleccione **Bias Report** (Informe de sesgos).
- En **Analysis name** (Nombre del análisis), ingrese “fraud bias by age”.
- En **Select the column your model predicts (target)** (Seleccionar la columna que predice su modelo [objetivo]), seleccione **fraud**.
- En **Is your predicted column a value or threshold?** (¿La columna de predicción es un valor o un umbral?), seleccione la opción **value**.
- En **Predicted value(s)** (Valor[es] predecido[s])**, ingrese **1**.
- En **Select the column to analyze for bias** (Seleccionar la columna para analizar sesgos), seleccione **customer_age**.

1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Check for bias</span> (Buscar sesgos).

Una vez que termine el trabajo, vea las métricas obtenidas. Tome nota si hay algún sesgo y planifique los pasos de procesamiento que desee realizar en la columna que ha analizado. 

1. Para cualquier análisis que desee guardar, seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Save</span> (Guardar).

Puede repetir estos pasos para cualquier columna que desee analizar en busca de sesgos.

Para continuar con este laboratorio, regrese a la <a href="#task1-3-continue" target="_self">Tarea 1.3</a>.

<a name="task1-4-1" id="task1-4-1"></a>

### Apéndice: preparar datos en SageMaker Data Wrangler (Tarea 1.4)

Combine los conjuntos de datos, únalos en SageMaker Data Wrangler con **policy_id**.

Con SageMaker Data Wrangler, puede unir datos en cualquier momento del flujo. Puede completar la preparación de los datos en los archivos individuales antes de unirlos, o puede transformar las funciones después de la unión. Un flujo de SageMaker Data Wrangler es flexible.

Si no ha unido las tablas en la tarea 1.3, los siguientes lo guiarán a través del proceso.

1. Regrese a la vista **Data flow** (Flujo de datos), seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.
1. Seleccione el símbolo **+** que se encuentra junto al ícono de **Data types** (Tipos de datos) y, en el menú contextual, seleccione **Join** (Unir).

SageMaker Data Wrangler mostrará la página **Join** (Unir).

1. Seleccione el segundo ícono **Data types** (Tipos de datos).
1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span> (Configurar).
1. En **Join Type** (Tipo de unión), seleccione **Inner** (Interna).
1. En la sección **Columns** (Columnas):

    - En **Left** (Izquierda), seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.
    
    - En **Right** (Derecha), seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>.

1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span> (Previsualizar).

1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span> (Agregar).

Con las tablas de datos unidas, transforme los datos combinados.

1. Regrese a la vista **Data flow** (Flujo de datos), seleccione **&lt; Data flow** (&lt; Flujo de datos), que se encuentra en la sección superior izquierda de la pestaña *CapstoneDataWrangler.flow*.
1. Seleccione el símbolo **+** que se encuentra junto al ícono de **Join** (Unir) y, en el menú contextual, seleccione **Add transform** (Agregar transformación).

Este menú permite agregar múltiples transformaciones a los conjuntos de datos. En la sección izquierda del menú de transformación se muestra una vista previa del conjunto de datos.

Agregue los siguientes pasos de transformación al flujo de SageMaker Data Wrangler:
- Codificación categórica (Codificación one-hot): **authorities_contacted**, **collision_type**, **customer_gender**, **driver_relationship**, **incident_type** y **policy_state** con una estrategia de gestión de valores no válidos **Skip** (Omitir), seleccione el estilo de salida **Columns** (Columnas).
- Codificación categórica (Codificación ordinal): **customer_education**, **incident_severity**, **police_report_available** y **policy_liability** con una estrategia de gestión de valores no válidos **Skip** (Omitir).
- Analizar columna como tipo: **vehicle_claim** y **total_claim_amount** de **Float** (Flotante) a **Long** (Larga).
- Administrar columnas (Drop column [Eliminar columna]): **customer_zip** y **policy_id_1**.
- Administrar columnas (Mover columna): **fraud** (usar **Move to start** [Mover para comenzar]).
- Administrar columnas (Rename column [Cambiar nombre de columna]): reemplazar el símbolo **/** de **collision_type_N/A** y **driver_relationship_N/A** con un **_**.
- Administrar columnas (Rename column [Cambiar nombre de columna]): cambiar el nombre **policy_id_0** a **policy_id**.

Si algún nombre de columna tienen un carácter **/**, cambie el nombre de la columna para reemplazar **/** por **_**. Si algún nombre de columna tienen espacio en blanco, cambie el nombre de la columna para reemplazar el espacio en blanco por **_**. Por ejemplo, se debe cambiar el nombre de cualquier columna creada con la codificación one-hot que tenga **N/A** como valor. SageMaker Feature Store no acepta columnas que tengan una **/** o un espacio.

Cuando haya transformado sus datos y pueda comenzar a entrenar al modelo, puede continuar con la siguiente tarea. Siempre puede volver regresar a este flujo y hacer cambios de acuerdo a sus hallazgos durante el entrenamiento y el ajuste.


Para continuar con este laboratorio, regrese a la Tarea <a href="#task1-4-continue" target="_self">1.4</a>.

<a name="task1-4-2" id="task1-4-2"></a>

### Apéndice: importar un conjunto de datos procesados de ejemplo (Tarea 1.4)

Si se traba durante el procesamiento previo o desea cargar un conjunto de datos que ya se ha procesado, acceda a los datos procesados almacenados en la carpeta de datos del bucket de S3.

In [ ]:
#processed-data-import
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims_customer.csv")
df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
df_processed.head()

Para continuar con este laboratorio, regrese a la Tarea <a href="#task1-4-continue" target="_self">1.4</a>.

<a name="task2-1" id="task2-1"></a>

### Apéndice: crear un grupo de funciones con la opción Explore to (Explorar a) (Tarea 2.1)

SageMaker Data Wrangler puede exportar datos al almacén de funciones de SageMaker. Crea un cuaderno con todo el código necesario para configurar un grupo de funciones e ingerir los datos transformados en el grupo de funciones.

1. Seleccione **Add step** (Agregar paso).

1. Seleccione **Custom transform** (Transformación personalizada).

1. En **Name** (Nombre), ingresar `event_time`. 

1. Seleccionar **Python (PySpark)** si aún no está seleccionado.

1. En **Your custom transform** (Su transformación personalizada), ingrese lo siguiente:


In [ ]:
import time
import datetime
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


1. Seleccione <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span> (Previsualizar).

1. Seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span> (Agregar).

Esto agrega **event_time** como una columna a su conjunto de datos. SageMaker Feature Store requiere un **event_time** y una identificación única de **record**. Utilice **policy_id** como su identificación de **record**.

1. Para regresar al flujo de datos, seleccione el ícono **&lt;Data flow** (&lt;Flujo de datos).

1. Seleccione el ícono **+** que se encuentra junto a sus transformaciones en SageMaker Data Wrangler.

1. Seleccione **Export to** (Exportar a).

1. Seleccione **SageMaker Feature Store (via Jupyter Notebook)** (Almacén de funciones de SageMaker [a través del cuaderno de Jupyter]).

Se abrirá un nuevo cuaderno.

1. En la primera celda, cambie las siguientes variables:
- En **record_identifier_feature_name**, reemplace **None** por `"policy_id"`. Si ha unido las tablas de clientes y reclamaciones y no eliminó la segunda columna **policy_id**, es posible que necesite reemplazar **None** por `"policy_id_0"`. No cambie el valor **None** después del enunciado **if**.
- En **event_time_feature_name**, reemplace **None** por `"event_time"`. No cambie el valor **None** después del enunciado **if**.

**Expected output** (Resultado esperado): cuando termine de editar la celda, se debería ver como el siguiente ejemplo:


In [ ]:
record_identifier_feature_name = "policy_id"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "event_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


2. Recorra todas las celdas para crear una definición de función, un grupo de funciones, e ingerir los datos transformados en el grupo de funciones a través de un trabajo de procesamiento. 

Cuando las celdas estén completas, su almacén de funciones estará listo para su uso.

Para continuar con este laboratorio, regrese a la <a href="#task2-1-continue" target="_self">Tarea 2.1</a>.

<a name="task2-2" id="task2-2"></a>

### Apéndice: extraer registros de un almacén de funciones sin conexión con Athena (Tarea 2.2)

Configure una consulta de Athena con **athena_query**. A continuación, configure el **query_string**. Finalmente, ejecute la consulta y vea una muestra de los resultados.

In [ ]:
#configure-and-run-athena-query
try:
    # If there is a feature group, get the name
    feature_group_name = sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()['FeatureGroupSummaries'][0]['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

    # Confirm the Athena settings are configured
    try:
        boto3.client('athena').update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'EnforceWorkGroupConfiguration':False
            }
        )
    except Exception:
        pass
    
    # Configure the query
    query = feature_group.athena_query()
    table = query.table_name
    query_string = f'SELECT * FROM "{table}" '
    output_location = f"s3://{sagemaker_session.default_bucket()}/query_results/"
    print(f"Athena query output location: \n{output_location}")

    # Run the query
    query.run(query_string=query_string, output_location=output_location)
    query.wait()
    df_feature_store = query.as_dataframe()
    
    # Wait for data to appear in the feature group
    attempts = 0
    while len(df_feature_store.index) == 0 and attempts < 30:
        print("Waiting for feature group to populate...")
        time.sleep(60)
        # Rerun the query
        query.run(query_string=query_string, output_location=output_location)
        query.wait()
        df_feature_store = query.as_dataframe()
        # Increment the attempts
        attempts += 1
    if len(df_feature_store.index) != 0:
        print("The feature group is populated.")
except IndexError as e:
    # If there is no feature group, thrown an error
    print("No feature groups were found. Please create a feature group.")

Después de ejecutar el bloque de código anterior, el registro de consultas de Amazon Athena se guarda en el bucket de Amazon S3 que tiene un nombre que comienza con *sagemaker*. El objeto de la consulta se guarda en un directorio llamado **query_results**.

Para continuar con este laboratorio, regrese a la <a href="#task2-2-continue" target="_self">Tarea 2.2</a>.

<a name="task3-1" id="task3-1"></a>

### Apéndice: nombrar un experimento y una ejecución (Tarea 3.1).

Para crear un experimento, utilice la biblioteca **sagemaker.experiments.run**. Configure **experiment_name**, **run_name** y **description**.

In [ ]:
from time import gmtime, strftime

#create experiment and run-names
create_date = strftime("%m%d%H%M")
capstone_experiment_name="capstone-experiment-{}".format(create_date)
capstone_run_name = "lab-capstone-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-capstone', 'Value': 'lab-capstone-run'}]

# provide a description
description="Using SM Experiments with the Auto dataset."

print(f"Experiment name - {capstone_experiment_name},  run name - {capstone_run_name}")

Para continuar con este laboratorio, regrese a la <a href="#task3-1-continue" target="_self">Tarea 3.1</a>.

<a name="task3-2" id="task3-2"></a>

### Apéndice: dividir datos en conjuntos de datos de entrenamiento, prueba y validación (Tarea 3.2)

Para dividir los datos, utilice **np.split** y especifique cómo quiere dividir los datos. A continuación, cree archivos CSV y cárguelos en el bucket de S3. Luego, configure las entradas del entrenamiento. Finalmente, cree la variable **data_inputs**. Utilice data_inputs a lo largo del desafío para especificar los conjuntos de datos de entrenamiento y validación cuando entrene al modelo.

In [ ]:
#train-validation-test-split
try:
    # If there is a feature group, use it
    df_feature_store = df_feature_store.iloc[: , :-4]
    df_processed_pre_split = df_feature_store
    print("Using the records from the feature group")
except NameError:
    # If there is no feature group, use the processed dataset
    df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
    df_processed_pre_split = df_processed
    print("Using the processed records from Amazon S3")

# Split the data into train, validation, and test datasets
train_data, validation_data, test_data = np.split(
    df_processed_pre_split.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_processed_pre_split)), int(0.9 * len(df_processed_pre_split))],
)

# Create the CSV files and upload them to your default bucket
train_data.to_csv("train_data.csv", index=False, header=False)
validation_data.to_csv("validation_data.csv", index=False, header=False)
test_data.to_csv("test_data.csv", index=False, header=False)

train_path = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
validation_path = S3Uploader.upload("validation_data.csv", "s3://{}/{}".format(bucket, prefix))
test_path = S3Uploader.upload("test_data.csv", "s3://{}/{}".format(bucket, prefix))

# Set the training inputs
train_input = TrainingInput(train_path, content_type="text/csv")
validation_input = TrainingInput(validation_path, content_type="text/csv")
test_input = TrainingInput(test_path, content_type="text/csv")

data_inputs = {
    "train": train_input,
    "validation": validation_input
}

Para continuar con este laboratorio, regrese a la <a href="#task3-2-continue" target="_self">Tarea 3.2</a>.

<a name="task3-3-1" id="task3-3-1"></a>

### Apéndice: configurar y ejecutar un trabajo de entrenamiento básico (Tarea 3.3)

Si desea comenzar con un trabajo de entrenamiento básico, utilice el contenedor básico **XGBoost**. A continuación, configure un estimador, teniendo en cuenta el contenedor y el rol que desea utilizar. Cuando esas configuraciones están listas, puede seleccionar los hiperparámetros. Puede utilizar los valores predeterminados que se proporcionan en el siguiente código, o puede editarlos en función de sus hallazgos durante la preparación de los datos.

Para ejecutar un trabajo de entrenamiento, llame a **fit()**, configure las entradas como su variable **data_inputs** y ajuste las configuraciones de **run_name** y **experiment_name**.

In [ ]:

from sagemaker import image_uris
#train-model
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8

hyperparameters = {
    "eta":eta,
    "gamma":gamma,
    "max_depth":max_depth,
    "min_child_weight":min_child_weight,
    "num_round":num_round,
    "objective":objective,
    "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role,    
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags
)

with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    )        

Para continuar con este laboratorio, regrese a la <a href="#task3-3-continue" target="_self">Tarea 3.3</a>.

<a name="task3-3-2" id="task3-3-2"></a>

### Apéndice: configurar y ejecutar un trabajo de entrenamiento con el depurador de SageMaker habilitado y analizar los informes (Tarea 3.3).

El depurador de SageMaker lo ayuda a encontrar rápidamente informes adicionales que pueden informar sobre el ajuste de hiperparámetros, lo que le permite ahorrar tiempo cuando comienza a ejecutar más trabajos de entrenamiento con los rangos de hiperparámetros. Para habilitar el depurador, configure **DebuggerHookConfig** y **rules**.

In [ ]:
#enable-debugger
# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# Set the hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=300
objective='binary:logistic'
subsample=0.7
        
hyperparameters = {
        "eta":eta,
        "gamma":gamma,
        "max_depth":max_depth,
        "min_child_weight":min_child_weight,
        "num_round":num_round,
        "objective":objective,
        "subsample":subsample
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags,

    #Set the Debugger Hook Config
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
        ),
        #Set the Debugger Profiler Configuration
        profiler_config = ProfilerConfig(
            system_monitor_interval_millis=500,
            framework_profile_params=FrameworkProfile()
    ),
        #Configure the Debugger Rule Object
        rules = [
            ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
            Rule.sagemaker(rule_configs.create_xgboost_report()),  
            Rule.sagemaker(rule_configs.overfit()),
            Rule.sagemaker(rule_configs.overtraining()),
            Rule.sagemaker(rule_configs.loss_not_decreasing(),
                rule_parameters={
                    "collection_names": "metrics",
                    "num_steps": str(save_interval * 2),
                }
            )
    ]
)
with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)
# Train the model
xgb.fit(
    inputs = data_inputs
) 

Para continuar con este laboratorio, regrese a la <a href="#task3-3-continue" target="_self">Tarea 3.3</a>.

<a name="task3-4" id="task3-4"></a>

### Apéndice: configurar los rangos de los hiperparámetros de entrenamiento (Tarea 3.4)

Ahora que ha entrenado al menos un modelo, puede utilizar lo que ha aprendido del procesamiento de datos y el depurador de SageMaker para informar qué rangos selecciona para sus hiperparámetros. Edite los siguientes rangos de hiperparámetros y ejecute el trabajo de ajuste.

In [ ]:
#tune-model
# Setup the hyperparameter ranges
hyperparameter_ranges = {
    "alpha": ContinuousParameter(0, 2),
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "min_child_weight": ContinuousParameter(1, 10),
    "num_round": IntegerParameter(100, 1000)
}

# Define the target metric and the objective type (max/min)
objective_metric_name = "validation:auc"
objective_type="Maximize"

# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type="Auto"
)

# Tune the model
tuner.fit(
    inputs = data_inputs
)

Para continuar con este laboratorio, regrese a la <a href="#task3-4-continue" target="_self">Tarea 3.4</a>.

<a name="task4-1" id="task4-1"></a>

### Apéndice: crear un modelo (Tarea 4.1)

Cree un modelo XGBoost, llamando **create_model** con el **model_name**, **role** y **container_def** que usted defina.

In [ ]:
#model-configurations
model_name = "capstone-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

Para continuar con este laboratorio, regrese a la <a href="#task4-1-continue" target="_self">Tarea 4.1</a>.

<a name="task4-2" id="task4-2"></a>

### Apéndice: crear una configuración del modelo de SageMaker Clarify (Tarea 4.2)

Cree una configuración del modelo de SageMaker Clarify con **SageMakerClarifyProcessor**. Configure **instance_count** e **instance_type**. Utilice el **rol** y la **sesión** creados al comienzo del curso de culminación.

In [ ]:
#define-clarify-processor
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sagemaker_session
)

Para continuar con este laboratorio, regrese a la <a href="#task4-2-continue" target="_self">Tarea 4.2</a>.

<a name="task4-3" id="task4-3"></a>

### Apéndice: crear una configuración de sesgos de SageMaker Clarify (Tarea 4.3)

Para crear una configuración de sesgos de SageMaker Clarify, seleccione una ruta de salida para los datos, configure la ruta de entrada para el trabajo de entrenamiento, además de **label **, **headers** y **dataset_type**.

A continuación, cree **ModelConfig** y **ModelPredictedLabelConfig**.

Finalmente, configure **BiasConfig** con los campos que quiere que SageMaker Clarify observe. Puede agregar o eliminar cualquier campo que le interese explorar en función de sus hallazgos iniciales.

In [ ]:
#define-data-config
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
#define-model-config
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
#define-label-config
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
#define-bias-config
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="customer_gender_female", facet_values_or_threshold=[0], group_name="customer_age"
)

Para continuar con este laboratorio, regrese a la <a href="#task4-3-continue" target="_self">Tarea 4.3</a>.

<a name="task4-4" id="task4-4"></a>

### Apéndice: ejecutar informes de sesgos, datos y modelos con SageMaker Clarify (Tarea 4.4)

Ahora que su trabajo de SageMaker Clarify está configurado, ejecute el trabajo a través de una llamada a **run_bias**.

In [ ]:
#run-bias-report
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

Para continuar con este laboratorio, regrese a la <a href="#task4-4-continue" target="_self">Tarea 4.4</a>.

<a name="task4-5" id="task4-5"></a>

### Apéndice: eliminar el desequilibrio (Tarea 4.5)

En este ejemplo, está realizando un sobre muestreo de **customer_gender_female** para reducir el sesgo en el conjunto de datos. Si encuentra otras funciones que contienen sesgos, también puede eliminar los desequilibrios en esas funciones. El **random_state** se ha establecido en `42`, pero puede cambiarlo.

In [ ]:
#display-summary
gender = train_data["customer_gender_female"]
gender.value_counts()

In [ ]:
#remove-imbalance
sm = SMOTE(random_state=42)
train_data_upsampled, gender_res = sm.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

Para continuar con este laboratorio, regrese a la <a href="#task4-5-continue" target="_self">Tarea 4.5</a>.

<a name="task4-6" id="task4-6"></a>

### Apéndice: volver a entrenar el modelo (Tarea 4.6)

Ha encontrado un desequilibrio y tiene un nuevo conjunto de datos de entrenamiento. Utilice este conjunto de datos y vuelva a entrenar el archivo. Para hacerlo, cargue el archivo nuevo y cree un nuevo estimador. A continuación, vuelva a entrenar los datos con **fit()**. En el siguiente código se incluyen varios hiperparámetros como muestra. Puede agregarlos, eliminarlos o ajustarlos según sea necesario durante el nuevo entrenamiento para hallar el mejor modelo.

In [ ]:
#upload-upsampled-csv
train_data_upsampled.to_csv("data/upsampled_train.csv", index=False, header=False)
retrain_path = S3Uploader.upload("data/upsampled_train.csv", "s3://{}/{}".format(bucket, prefix))
retrain_input = TrainingInput(retrain_path, content_type="text/csv")

retrain_data_inputs = {
    "train": retrain_input,
    "validation": validation_input
}

In [ ]:
#create-estimator
hyperparameters= {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "300",
}

xgb_retrained = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
)

In [ ]:
#retrain-upsampled-data
xgb_retrained.fit(
    inputs = retrain_data_inputs
) 

Para continuar con este laboratorio, regrese a la <a href="#task4-6-continue" target="_self">Tarea 4.6</a>.

<a name="task5-1" id="task5-1"></a>

### Apéndice: crear un trabajo de transformación por lotes (Tarea 5.1)

Utilice el estimador del modelo y cree un trabajo de transformación por lotes con **transformer** (transformador). Configure la estrategia en **MultiRecord** para incrementar la eficacia del procesamiento. A continuación, introduzca su **test_path** y aguarde que se ejecute la interferencia.

In [ ]:
#create-batch-transformer
# Use the retrained model if it exists, otherwise, use the original model
try:
    model = xgb_retrained
except NameError:
    model = xgb

# Create the transformer
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/".format(bucket, prefix)
)

In [ ]:
#run-batch-transform-job
test_data_batch = test_data.drop("fraud", axis=1)
test_data_batch.to_csv("test_data_batch.csv", index=False, header=False)
test_path_batch = S3Uploader.upload("test_data_batch.csv", "s3://{}/{}".format(bucket, prefix))

transformer.transform(test_path_batch, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

Para continuar con este laboratorio, regrese a la <a href="#task5-1-continue" target="_self">Tarea 5.1</a>.

<a name="task5-2" id="task5-2"></a>

### Apéndice: ver los datos de predicción y exactitud de un trabajo de transformación por lotes (Tarea 5.2)

Cuando finalice el trabajo de transformación por lotes, vea los datos de predicción almacenados en Amazon S3. Puede referenciar la ruta de salida que configuró en el **transformer** (transformador) hacer una muestra de los datos.

En esta salida, la predicción **fraud** (fraude) se añade al final de cada registro.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
test_data = pd.read_csv("test_data_batch.csv.out")
test_data.head()

Para continuar con este laboratorio, regrese a la <a href="#task5-2-continue" target="_self">Tarea 5.2</a>.

<a name="task5-3" id="task5-3"></a>

### Apéndice: realizar una limpieza de las instancias de SageMaker en SageMaker Studio (Tarea 5.3)

A medida que desarrolle modelos en SageMaker Studio, verifique de forma periódica si hay alguna instancia que quiera limpiar. En ese caso, puede apagar las instancias dentro de SageMaker Studio.

1. En la barra de menú, seleccione el ícono **Running Terminals and Kernels** (Terminales y kernels en ejecución) (un círculo con un cuadrado en el medio).

1. Si aún quedan instancias abiertas, a la derecha de cada tipo de instancia, seleccione el ícono **Shut down** (Apagar). 

Puede ver las aplicaciones que se están ejecutando en cada instancia para confirmar cuáles quiere cerrar.

1. Si aparece una ventana emergente, seleccione **Shut down all** (Apagar todo).

1. Seleccione el ícono **Refresh List** (Actualizar lista) de forma periódica hasta que la instancia ya no aparezca en la lista. Una instancia puede tardar entre 2 y 5 minutos para apagarse.

No es necesario que apague la instancia que está usando el cuaderno**capstone.ipynb**.

Para continuar con este laboratorio, regrese a la <a href="#task5-3-continue" target="_self">Tarea 5.3</a>.

<a name="task6-1" id="task6-1"></a>

### Apéndice: configurar una canalización (Tarea 6.1)

Para crear una canalización, defina cada paso del proceso de canalización y luego ejecútelo.

En este ejemplo, se crean los siguientes pasos:
- **AutoModelProcess**: un paso de **Procesamiento** que extrae el archivo .csv y lo divide en conjuntos de datos de entrenamiento, prueba y validación.
- **AutoHyperParameterTuning**: un paso de **Ajuste** que utiliza una amplia gama de hiperparámetros y ajusta el modelo.
- **AutoEvalBestModel**: un paso de **Procesamiento** que crea un informe de evaluación para describir al mejor modelo.
- **CheckAUCScoreAutoEvaluation**: un paso de **Condición** que evalúa los modelos basados en una métrica de evaluación. 
- **AutoCreateModel**: un paso de **Modelo** que crea el modelo.
- **RegisterAutoModel-RegisterModel**: un paso de **Registro de modelo** que registra el modelo.
- **AutoModelConfigFile**: un paso de **Procesamiento** que crea un informe de sesgos.
- **AutoTransform**: un paso de **Transformación** que ejecuta un trabajo de transformación por lotes.
- **ClarifyProcessingStep**: un paso de **Procesamiento** que ejecuta un trabajo de SageMaker Clarify.

Si se traba en algún punto mientras crea la canalización, puede personalizar el siguiente código o utilizarlo como guía mientras crea su propia canalización.

In [ ]:
#run-pipeline
# Set the variables
model_name = "Auto-model"
sklearn_processor_version="0.23-1"
model_package_group_name="AutoModelPackageGroup"
pipeline_name= "AutoModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

# Upload files to the default S3 bucket
s3_client.put_object(Bucket=bucket,Key='data/')
s3_client.put_object(Bucket=bucket,Key='input/code/')
s3_client.upload_file(Filename="data/batch_data.csv", Bucket=bucket, Key="data/batch_data.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key="data/claims_customer.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="pipelines/evaluate.py", Bucket=bucket, Key="input/code/evaluate.py")
s3_client.upload_file(Filename="pipelines/generate_config.py", Bucket=bucket, Key="input/code/generate_config.py")
s3_client.upload_file(Filename="pipelines/preprocess.py", Bucket=bucket, Key="input/code/preprocess.py")

# Configure important settings. Change the input_data if you want to
# use a file other than the claims_customer.csv and batch_data.csv files.
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
        name="TrainingInstanceType",
        default_value="ml.m5.xlarge"
)
input_data = ParameterString(
        name="InputData",
        default_value="s3://{}/data/claims_customer.csv".format(bucket), 
)
batch_data = ParameterString(
        name="BatchData",
        default_value="s3://{}/data/batch_data.csv".format(bucket),
)

# Run a scikit-learn script to do data processing on SageMaker using 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version=sklearn_processor_version,
        instance_type=processing_instance_type.default_value, 
        instance_count=processing_instance_count,
        sagemaker_session=sagemaker_session,
        role=role,
)

# Configure the processing step to pull in the input_data
step_process = ProcessingStep(
        name="AutoModelProcess",
        processor=sklearn_processor,
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train",\
                             destination=f"s3://{bucket}/output/train" ),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",\
                            destination=f"s3://{bucket}/output/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test",\
                            destination=f"s3://{bucket}/output/test"),
            ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch",\
                            destination=f"s3://{bucket}/data/batch"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline",\
                            destination=f"s3://{bucket}/input/baseline")
        ],
        code=f"s3://{bucket}/input/code/preprocess.py",
        job_arguments=["--input-data", input_data],
)

# Set up the model path, image uri, and hyperparameters for the estimator
model_path = f"s3://{bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1",
    py_version="py3",
    instance_type=training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=model_path,
    base_job_name=f"auto-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set the hyperparameter ranges for the tuning step and configure the tuning step
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "AutoHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs=2, max_parallel_jobs=2),
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# Configure the processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-auto-eval",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="AutoEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="AutoEvalBestModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",\
                            destination=f"s3://{bucket}/output/evaluation"),
    ],
    code=f"s3://{bucket}/input/code/evaluate.py",
    property_files=[evaluation_report],
)

# Configure model creation
model = Model(
    image_uri=image_uri,        
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name="AutoCreateModel",
    model=model,
    inputs=inputs,
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=clarify_image,
    role=role,
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session,
)

bias_report_output_path = f"s3://{bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
step_config_file = ProcessingStep(
    name="AutoModelConfigFile",
    processor=script_processor,
    code=f"s3://{bucket}/input/code/generate_config.py",
    job_arguments=["--modelname",step_create_model.properties.ModelName,"--bias-report-output-path",bias_report_output_path,"--clarify-instance-type",clarify_instance_type,\
                  "--default-bucket",bucket,"--num-baseline-samples","50","--instance-count","1"],
    depends_on= [step_create_model.name]
)

# Configure the step to perform a batch transform job
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line",
    accept="text/csv",    
    output_path=f"s3://{bucket}/AutoTransform"
)

step_transform = TransformStep(
    name="AutoTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data,content_type="text/csv",join_source="Input",split_type="Line")
)

# Configure the SageMaker Clarify processing step
analysis_config_path = f"s3://{bucket}/clarify-output/bias/analysis_config.json"

data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=f's3://{bucket}/output/train/train.csv', 
    s3_output_path=bias_report_output_path,
    label=0,
    headers=list(pd.read_csv("./data/claims_customer.csv", index_col=None).columns), #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
    dataset_type="text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type=clarify_instance_type,
    sagemaker_session=sagemaker_session,
)

config_input = ProcessingInput(
    input_name="analysis_config",
    source=analysis_config_path,
    destination="/opt/ml/processing/input/analysis_config",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_compression_type="None",
)

data_input = ProcessingInput(
    input_name="dataset",
    source=data_config.s3_data_input_path,
    destination="/opt/ml/processing/input/data",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type=data_config.s3_data_distribution_type,
    s3_compression_type=data_config.s3_compression_type,
)

result_output = ProcessingOutput(
    source="/opt/ml/processing/output",
    destination=data_config.s3_output_path,
    output_name="analysis_result",
    s3_upload_mode="EndOfJob",
)

step_clarify = ProcessingStep(
    name="ClarifyProcessingStep",
    processor=clarify_processor,
    inputs= [data_input, config_input],
    outputs=[result_output],
    depends_on = [step_config_file.name]
)

# Configure the model registration step
model_statistics = MetricsSource(
    s3_uri="s3://{}/output/evaluation/evaluation.json".format(bucket),
    content_type="application/json"
)
explainability = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
)

bias = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
) 

model_metrics = ModelMetrics(
    model_statistics=model_statistics,
    explainability=explainability,
    bias=bias
)

step_register = RegisterModel(
    name="RegisterAutoModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
)

# Create the model evaluation step
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value"
    ),
    right=0.75,
)

step_cond = ConditionStep(
    name="CheckAUCScoreAutoEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model,step_config_file,step_transform,step_clarify,step_register],
    else_steps=[],
)

# Define the pipeline
def get_pipeline(
    region,
    role=None,
    default_bucket=None,
    model_package_group_name="AutoModelPackageGroup",
    pipeline_name="AutoModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version=None
    ):
    """Gets a SageMaker ML Pipeline instance working with auto data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """

    # pipeline instance
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps=[step_process,step_tuning,step_eval,step_cond],
        sagemaker_session=sagemaker_session
    )
    return pipeline


# Create the pipeline
pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

pipeline.upsert(role_arn=role)

# Run the pipeline
RunPipeline = pipeline.start()

Para continuar con este laboratorio, regrese a la <a href="#task6-1-continue" target="_self">Tarea 6.1</a>.

<a name="task6-2" id="task6-2"></a>

### Apéndice: supervisar la canalización (Tarea 6.2)

Ahora que ya ha creado y ejecutado la canalización, supervísela. Puede ver el estado de la canalización en SageMaker Studio.

Si desea eliminar la canalización, puede hacerlo con **delete_pipeline**.

In [ ]:
#describe-pipeline
RunPipeline.describe()

In [ ]:
#list-pipeline-steps
RunPipeline.list_steps()

In [ ]:
#remove-pipeline
response = sagemaker_session.boto_session.client("sagemaker", region_name=region).delete_pipeline(PipelineName='AutoModelSMPipeline')
print(response)

Para continuar con este laboratorio, regrese a la <a href="#task6-2-continue" target="_self">Tarea 6.2</a>.